<a href="https://colab.research.google.com/github/PedroHenriqueFonsecaMelo/Exercicios_IA_Python_UMC_EngenhariaDeSoftware_6B/blob/main/Correcao_dos_exercicios_de_PedroHFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎓 Comparação de Técnicas de Codificação Categórica com SVM
Este notebook demonstra como aplicar e comparar **Label Encoding** e **One-Hot Encoding** em variáveis categóricas, com foco no uso de **SVM** para classificação.

Inclui exemplos com os datasets `iris`, `titanic` e `penguins`, todos acompanhados de explicações, visualizações e análises didáticas.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

### 🧠 Objetivo Geral
Este notebook tem como objetivo ensinar como aplicar diferentes técnicas de codificação de variáveis categóricas e entender suas implicações na modelagem com SVM (Support Vector Machine).

Você aprenderá a:
- Aplicar Label Encoding e One-Hot Encoding
- Entender quando cada abordagem é adequada
- Lidar com o formato esperado pelo SVM
- Interpretar erros comuns e como resolvê-los

## 🟢 Exercício 1 – Dataset `iris`
Neste exemplo, vamos aplicar **Label Encoding** e **One-Hot Encoding** no vetor alvo (`species`).
- Mostraremos por que One-Hot precisa ser reconvertido com `argmax()` antes de treinar um modelo `SVC`.
- Também compararemos visualmente os formatos.

In [ ]:
# Carregar dataset
iris = load_iris(as_frame=True)
X = iris.data
species = iris.target_names[iris.target]

In [ ]:
# Aplicando Label Encoding
le = LabelEncoder()
y_label = le.fit_transform(species)
print("Label Encoding (y):", y_label[:5])

Label Encoding (y): [0 0 0 0 0]


### ℹ️ Por que o SVM não aceita diretamente One-Hot Encoding no alvo (y)?
O modelo `SVC()` espera que o vetor alvo (`y`) seja **unidimensional**, contendo rótulos categóricos como inteiros ou strings.

Quando aplicamos One-Hot Encoding no `y`, ele se torna uma **matriz binária**, com múltiplas colunas. Isso quebra a expectativa do modelo.

**Solução:** usamos `np.argmax(y_oh, axis=1)` para reconverter cada vetor binário para o índice da classe (como `0`, `1`, `2`).

In [ ]:
# Aplicando One-Hot Encoding no target
enc = OneHotEncoder(sparse_output=False)
y_oh = enc.fit_transform(species.reshape(-1, 1))
print("One-Hot Encoding (y):\n", y_oh[:5])

One-Hot Encoding (y):
 [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
# Reconversão com argmax (para uso com SVC)
y_oh_labels = np.argmax(y_oh, axis=1)
print("Reconversão (y):", y_oh_labels[:5])

Reconversão (y): [0 0 0 0 0]


In [ ]:
# Treinar com Label Encoding
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.3, random_state=42)
model_label = SVC(kernel='linear')
model_label.fit(X_train, y_train)
acc_label = accuracy_score(y_test, model_label.predict(X_test))
print("Acurácia com Label Encoding:", round(acc_label, 2))

Acurácia com Label Encoding: 1.0


In [ ]:
# Treinar com One-Hot reconvertido
X_train, X_test, y_train, y_test = train_test_split(X, y_oh_labels, test_size=0.3, random_state=42)
model_oh = SVC(kernel='linear')
model_oh.fit(X_train, y_train)
acc_oh = accuracy_score(y_test, model_oh.predict(X_test))
print("Acurácia com One-Hot (convertido):", round(acc_oh, 2))

Acurácia com One-Hot (convertido): 1.0


## 🟡 Exercício 2 – Dataset `titanic`
Neste exemplo, aplicamos codificação nas features categóricas (`sex`, `embarked`, `class`) para prever `survived`.

- Usaremos Label Encoding e One-Hot Encoding nas features.
- Compararemos o desempenho no SVM.

In [ ]:
# Carregar dataset Titanic
df = sns.load_dataset('titanic').dropna(subset=['sex', 'embarked', 'class', 'age', 'survived'])
features = ['sex', 'embarked', 'class', 'age']
target = 'survived'

In [ ]:
# Label Encoding nas features
le_sex = LabelEncoder()
le_emb = LabelEncoder()
le_class = LabelEncoder()
df_label = df.copy()
df_label['sex'] = le_sex.fit_transform(df['sex'])
df_label['embarked'] = le_emb.fit_transform(df['embarked'])
df_label['class'] = le_class.fit_transform(df['class'])
X_label = df_label[features]
y = df_label[target]
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X_label, y, test_size=0.3, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = SVC(kernel='linear')
model.fit(X_train, y_train)
acc_label = accuracy_score(y_test, model.predict(X_test))
print("Acurácia com Label Encoding:", round(acc_label, 2))

Acurácia com Label Encoding: 0.75


In [ ]:
# One-Hot Encoding nas features
X_oh = pd.get_dummies(df[features])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X_oh, y, test_size=0.3, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = SVC(kernel='linear')
model.fit(X_train, y_train)
acc_oh = accuracy_score(y_test, model.predict(X_test))
print("Acurácia com One-Hot Encoding:", round(acc_oh, 2))

Acurácia com One-Hot Encoding: 0.75


## 🔴 Exercício 3 – Dataset `penguins`
Neste exemplo, comparamos Label Encoding e One-Hot Encoding tanto nas features quanto no alvo (`species`).

- Também demonstraremos por que One-Hot no alvo precisa ser reconvertido com `argmax()`.

In [ ]:
# Carregar dataset Penguins
penguins = sns.load_dataset('penguins').dropna()
cat_cols = ['island', 'sex']
num_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
le1, le2, le3 = LabelEncoder(), LabelEncoder(), LabelEncoder()
X_label = penguins[cat_cols + num_cols].copy()
X_label['island'] = le1.fit_transform(X_label['island'])
X_label['sex'] = le2.fit_transform(X_label['sex'])
y_label = le3.fit_transform(penguins['species'])

In [ ]:
# One-Hot no alvo (y), reconvertendo com argmax
ohe = OneHotEncoder(sparse_output=False)
y_oh = ohe.fit_transform(penguins[['species']])
y_oh_labels = np.argmax(y_oh, axis=1)
print("Exemplo One-Hot:")
print(y_oh[:5])
print("Reconversão:", y_oh_labels[:5])

Exemplo One-Hot:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Reconversão: [0 0 0 0 0]


In [ ]:
# SVM com Label Encoding
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X_label, y_label, test_size=0.3, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = SVC(kernel='linear')
model.fit(X_train, y_train)
acc_label = accuracy_score(y_test, model.predict(X_test))
print("Acurácia com Label Encoding:", round(acc_label, 2))

Acurácia com Label Encoding: 0.99


In [ ]:
# One-Hot nas features + reconvertido no y
X_oh = pd.get_dummies(penguins[cat_cols + num_cols])
X_train, X_test, y_train, y_test = train_test_split(X_oh, y_oh_labels, test_size=0.3, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = SVC(kernel='linear')
model.fit(X_train, y_train)
acc_oh = accuracy_score(y_test, model.predict(X_test))
print("Acurácia com One-Hot (convertido):", round(acc_oh, 2))

Acurácia com One-Hot (convertido): 1.0


## 🧪 Exercícios Propostos para Prática
A seguir, propomos três exercícios para reforçar o entendimento sobre codificação de variáveis categóricas e aplicação de SVM.

### 🔹 Exercício 4 – Dataset `tips`
- Carregue o dataset `tips` do seaborn.
- Use as colunas `sex`, `smoker`, `day`, `time` como features categóricas.
- Aplique Label Encoding e One-Hot Encoding para prever se a gorjeta foi maior que a média (coluna binária criada).
- Compare as acurácias dos dois modelos com SVM.

### 🔹 Exercício 5 – Dataset `diamonds`
- Use o dataset `diamonds` do seaborn.
- Converta a variável `cut` (categórica ordinal) com Label Encoding.
- Aplique One-Hot Encoding nas variáveis `color` e `clarity`.
- Crie uma variável alvo binária para prever se o preço é acima da mediana.
- Treine dois modelos SVM e compare os resultados.

### 🔹 Exercício 6 – Dataset `car_crashes`
- Use o dataset `car_crashes` do seaborn.
- Transforme o campo `abbrev` (sigla do estado) com Label e One-Hot.
- Crie uma variável alvo categórica que representa a taxa de mortes por 100 mil (acima ou abaixo da média).
- Aplique SVM com ambas as codificações e discuta os resultados.

💡 **Dica:** Use `StandardScaler()` para normalizar os dados antes de treinar o modelo SVM.

---
**Esses exercícios visam consolidar o entendimento da transformação de dados categóricos e avaliar seu impacto na modelagem preditiva.**

======================================/===================================

### 🔹 Exercício 4 – Dataset `tips`
- Carregue o dataset `tips` do seaborn.
- Use as colunas `sex`, `smoker`, `day`, `time` como features categóricas.
- Aplique Label Encoding e One-Hot Encoding para prever se a gorjeta foi maior que a média (coluna binária criada).
- Compare as acurácias dos dois modelos com SVM.

In [ ]:
tips = sns.load_dataset('tips').dropna()
features = ['sex', 'smoker', 'day','time']
target = 'day'

In [ ]:
# Label Encoding nas features
encoder = LabelEncoder()
df_label = tips.copy()

for col in features:
    df_label[str(col)] = encoder.fit_transform(tips[str(col)])

X_label = df_label[features]
y = df_label[target]

scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X_label, y, test_size=0.3, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

acc_label = accuracy_score(y_test, model.predict(X_test))
print("Acurácia com Label Encoding:", round(acc_label, 2))

Acurácia com Label Encoding: 1.0


In [ ]:
# One-Hot Encoding nas features
X_oh = pd.get_dummies(tips[features])
y = tips[target]

X_train, X_test, y_train, y_test = train_test_split(X_oh, y, test_size=0.3, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = SVC(kernel='linear')
model.fit(X_train, y_train)
acc_oh = accuracy_score(y_test, model.predict(X_test))
print("Acurácia com One-Hot Encoding:", round(acc_oh, 2))

Acurácia com One-Hot Encoding: 1.0


### 🔹 Exercício 5 – Dataset `diamonds`
- Use o dataset `diamonds` do seaborn.
- Converta a variável `cut` (categórica ordinal) com Label Encoding.
- Aplique One-Hot Encoding nas variáveis `color` e `clarity`.
- Crie uma variável alvo binária para prever se o preço é acima da mediana.
- Treine dois modelos SVM e compare os resultados.

In [ ]:
diamonds = sns.load_dataset('diamonds').dropna()

In [ ]:
label_encoder = LabelEncoder()


for col in diamonds:
    diamonds[str(col)] = label_encoder.fit_transform(diamonds[str(col)])


diamonds = pd.get_dummies(diamonds, columns=['color', 'clarity'])

diamonds.head()

,carat,cut,depth,table,price,x,y,z,color_0,color_1,...,color_5,color_6,clarity_0,clarity_1,clarity_2,clarity_3,clarity_4,clarity_5,clarity_6,clarity_7
0,3,2,83,30,0,20,26,24,False,True,...,False,False,False,False,False,True,False,False,False,False
1,1,3,66,90,0,14,12,12,False,True,...,False,False,False,False,True,False,False,False,False,False
2,3,1,37,115,1,30,35,12,False,True,...,False,False,False,False,False,False,True,False,False,False
3,9,3,92,60,2,45,51,44,False,False,...,True,False,False,False,False,False,False,True,False,False
4,11,1,101,60,3,59,63,56,False,False,...,False,True,False,False,False,True,False,False,False,False


In [ ]:
median_price = diamonds['price'].median()

diamonds['price_above_median'] = 0

for i, price in enumerate(diamonds['price']):
    if price > median_price:
        diamonds.at[i, 'price_above_median'] = 1
    else:
        diamonds.at[i, 'price_above_median'] = 0


In [ ]:
X = diamonds.drop(columns=['price', 'price_above_median'])
y = diamonds['price_above_median']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Modelo 1: SVM Linear
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train, y_train)
y_pred_linear = svm_linear.predict(X_test)
acc_linear = accuracy_score(y_test, y_pred_linear)

# Modelo 2: SVM RBF
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train, y_train)
y_pred_rbf = svm_rbf.predict(X_test)
acc_rbf = accuracy_score(y_test, y_pred_rbf)

print(f'Acurácia SVM Linear: {acc_linear:.4f}')
print(f'Acurácia SVM RBF: {acc_rbf:.4f}')


Acurácia SVM Linear: 0.9765
Acurácia SVM RBF: 0.9535
